In [1]:
import os, os.path
import numpy as np
import pandas as pd
import data_structures as ds
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
import model_afolu as ma
import model_circular_economy as mc
import model_ippu as mi
import model_socioeconomic as ms

importlib.reload(ds)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(ma)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(ms)

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:684: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:684: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")


<module 'model_socioeconomic' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_socioeconomic.py'>

In [228]:

#df_afolu_trajectories["frac_waso_recycled_paper"] = 1
#df_afolu_trajectories["timespan_ippu_average_lifetime_of_housing"] = 10
print(df_afolu_trajectories.shape) 


(36, 1591)


In [74]:
#[x for x in df_afolu_trajectories.columns if "recy" in x and "paper" in x]
#df_afolu_trajectories["frac_waso_recycled_paper"]
#sa.model_attributes.get_standard_variables(df_project_ippu, model_ippu.modvar_ippu_qty_total_production).head()
df_project_circ[[x for x in df_project_circ.columns if "wood"]]


,emission_co2e_ch4_waso_landfilled_wood,emission_co2e_ch4_waso_open_dump_wood,qty_waso_incinerated_wood_tonne,qty_waso_landfilled_wood_tonne,qty_waso_open_dump_wood_tonne,qty_waso_recycled_wood_tonne,qty_waso_total_wood_produced_tonne
0,0.000150,0.000010,229.052558,1446.647735,723.323868,170.376072,2581.455630
1,0.000166,0.000011,231.925223,1464.790880,732.395440,172.512845,2613.830978
2,0.000181,0.000012,234.649943,1481.999642,740.999821,174.539572,2644.538976
3,0.000197,0.000013,237.225933,1498.269049,749.134524,176.455670,2673.570751
4,0.000212,0.000014,239.721375,1514.029737,757.014868,178.311853,2701.694748
5,0.000226,0.000015,242.145486,1529.339911,764.669956,180.114979,2729.014831
6,0.000241,0.000016,244.540355,1544.465403,772.232701,181.896354,2756.005358
7,0.000256,0.000017,246.894044,1559.330807,779.665403,183.647097,2782.531775
8,0.000270,0.000018,249.225007,1574.052675,787.026337,185.380936,2808.802060
9,0.000284,0.000019,251.532861,1588.628593,794.314297,187.097586,2834.811908


In [151]:
df_tmp = pd.read_csv("/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/ingestion/FAOSTAT/Forestry_E_All_Data_(Normalized)/Forestry_E_All_Data_(Normalized)_utf8.csv")

df_tmp[
    df_tmp["Area"].isin(["Costa Rica"]) & 
    df_tmp["Year"].isin([2019]) & 
    df_tmp["Unit"].isin(["m3"]) &
    df_tmp["Element"].isin(["Production"])
]

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
372113,48,Costa Rica,1861,Roundwood,5516,Production,2019,2019,m3,4478540,Im
372385,48,Costa Rica,1862,"Roundwood, coniferous",5516,Production,2019,2019,m3,100000,Im
372461,48,Costa Rica,1863,"Roundwood, non-coniferous",5516,Production,2019,2019,m3,4378540,Im
372537,48,Costa Rica,1864,Wood fuel,5516,Production,2019,2019,m3,3257677,Im
372657,48,Costa Rica,1627,"Wood fuel, coniferous",5516,Production,2019,2019,m3,0,F
372721,48,Costa Rica,1628,"Wood fuel, non-coniferous",5516,Production,2019,2019,m3,3257677,F
372837,48,Costa Rica,1865,Industrial roundwood,5516,Production,2019,2019,m3,1220863,Im
373080,48,Costa Rica,1866,"Industrial roundwood, coniferous",5516,Production,2019,2019,m3,100000,Im
373372,48,Costa Rica,1867,"Industrial roundwood, non-coniferous",5516,Production,2019,2019,m3,1120863,Im
373704,48,Costa Rica,1868,Sawlogs and veneer logs,5516,Production,2019,2019,m3,974863,Im


In [137]:
10000000/1.5

6666666.666666667

In [174]:
#df_project_afolu[[x for x in df_project_afolu.columns if "emission_co2e_subsector_total" in x]]
importlib.reload(mi)
#importlib.reload(me)
importlib.reload(ma)
df_afolu_trajectories = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))
model_afolu = ma.AFOLU(sa.model_attributes)
model_circ = mc.CircularEconomy(sa.model_attributes)
model_ippu = mi.IPPU(sa.model_attributes)
warnings.filterwarnings("ignore")
t0 = time.time()#
#df_afolu_trajectories["timespan_ippu_average_lifetime_of_housing"] = 70
#df_afolu_trajectories["dem_ippu_average_materials_tonne_per_household_wood"] = [0.1, 0.2, 0.3, 0.4, 0.9, 1.2] + [1.2 for x in range(30)]
#df_afolu_trajectories["dem_ippu_average_materials_tonne_per_household_cement"] = np.array([1, 0.9, 0.8, 0.7] + [0 for x in range(32)]) 
#df_afolu_trajectories["prodinit_ippu_cement_tonne"] = 10
#df_afolu_trajectories["frac_scoe_heat_energy_residential_solid_biomass"] = [0.13, 0.12,0.11,0.1, 0.09, 0.05, 0.01] + [0.01 for x in range(29) ]
df_project_afolu = model_afolu.project(df_afolu_trajectories)
t1 = time.time()
#print(t1 - t0)
#df_project_circ = model_circ.project(df_afolu_trajectories)
#df_ippu_in = sf.merge_output_df_list([df_afolu_trajectories, df_project_circ], sa.model_attributes, "concatenate")

#df_project_ippu = model_ippu.project(df_ippu_in)
#sa.model_attributes.get_standard_variables(df_project_ippu, model_ippu.modvar_ippu_qty_total_production)
#df_project_afolu[[x for x in df_project_afolu if "emission_co2e_n2o_soil" in x]]#/.sum(axis = 1)
df_project_afolu[[x for x in df_project_afolu if "frst" in x and "emission" in x]]
#print(df_project_afolu["emission_co2e_co2_frst_harvested_wood_products"])


,emission_co2e_co2_frst_sequestration_mangroves,emission_co2e_co2_frst_sequestration_primary,emission_co2e_co2_frst_sequestration_secondary,emission_co2e_ch4_frst_methane_mangroves,emission_co2e_ch4_frst_methane_primary,emission_co2e_ch4_frst_methane_secondary,emission_co2e_co2_frst_forest_fires,emission_co2e_co2_frst_harvested_wood_products,emission_co2e_subsector_total_frst
0,-0.098780,-5.237938,-1.105878,0.001312,0.060891,0.025712,0.000180,-0.845928,-7.200430
1,-0.098629,-5.223194,-1.093686,0.001310,0.060720,0.025428,0.000178,-0.962952,-7.290823
2,-0.098375,-5.208907,-1.078163,0.001307,0.060554,0.025067,0.000176,-1.124725,-7.423065
3,-0.098081,-5.195096,-1.061796,0.001303,0.060393,0.024687,0.000174,-1.248971,-7.517387
4,-0.097764,-5.181750,-1.045307,0.001299,0.060238,0.024303,0.000172,-1.338532,-7.577342
5,-0.097441,-5.168867,-1.029302,0.001295,0.060088,0.023931,0.000170,-1.440880,-7.651006
6,-0.097092,-5.156436,-1.012820,0.001290,0.059944,0.023548,0.000167,-1.537841,-7.719241
7,-0.096713,-5.144451,-0.995659,0.001285,0.059804,0.023149,0.000165,-1.643765,-7.796186
8,-0.096293,-5.132905,-0.977297,0.001279,0.059670,0.022722,0.000162,-1.754190,-7.876852
9,-0.095844,-5.121791,-0.958192,0.001273,0.059541,0.022278,0.000160,-1.875013,-7.967588


In [290]:

df_afolu_trajectories["timespan_ippu_average_lifetime_of_housing"] = 10
df_afolu_trajectories["dem_ippu_average_materials_tonne_per_household_wood"] = [0.1, 0.2, 0.3, 0.4, 0.9, 1.2] + [1.2 for x in range(30)]
#df_afolu_trajectories["dem_ippu_average_materials_tonne_per_household_cement"] = np.array([1, 0.9, 0.8, 0.7] + [0 for x in range(32)]) 
#df_afolu_trajectories["prodinit_ippu_cement_tonne"] = 10
#df_afolu_trajectories["frac_scoe_heat_energy_residential_solid_biomass"] = [0.13, 0.12,0.11,0.1, 0.09, 0.05, 0.01] + [0.01 for x in range(29) ]
df_project_afolu = model_afolu.project(df_afolu_trajectories)

In [292]:
importlib.reload(ds)
importlib.reload(ma)

model_afolu = ma.AFOLU(sa.model_attributes)
#model_circ = mc.CircularEconomy(sa.model_attributes)
#model_ippu = mi.IPPU(sa.model_attributes)
warnings.filterwarnings("ignore")
df_project_afolu = model_afolu.project(df_afolu_trajectories)

df_project_afolu[[x for x in df_project_afolu if "emission_co2e_n2o_soil" in x]].sum(axis = 1)
df_project_afolu[[x for x in df_project_afolu if "frst" in x and "emission" in x]]#.sum(axis = 1)

,emission_co2e_co2_frst_sequestration_mangroves,emission_co2e_co2_frst_sequestration_primary,emission_co2e_co2_frst_sequestration_secondary,emission_co2e_ch4_frst_methane_mangroves,emission_co2e_ch4_frst_methane_primary,emission_co2e_ch4_frst_methane_secondary,emission_co2e_co2_frst_forest_fires,emission_co2e_co2_frst_harvested_wood_products,emission_co2e_subsector_total_frst
0,-0.098780,-5.237938,-1.105878,0.001312,0.060891,0.025712,0.000180,0.037583,-6.316919
1,-0.098629,-5.223194,-1.093686,0.001310,0.060720,0.025428,0.000178,0.000000,-6.327872
2,-0.098375,-5.208907,-1.078163,0.001307,0.060554,0.025067,0.000176,0.039608,-6.258733
3,-0.098081,-5.195096,-1.061796,0.001303,0.060393,0.024687,0.000174,0.020801,-6.247615
4,-0.097764,-5.181750,-1.045307,0.001299,0.060238,0.024303,0.000172,0.037893,-6.200917
5,-0.097441,-5.168867,-1.029302,0.001295,0.060088,0.023931,0.000170,0.018012,-6.192115
6,-0.097092,-5.156436,-1.012820,0.001290,0.059944,0.023548,0.000167,0.038186,-6.143213
7,-0.096713,-5.144451,-0.995659,0.001285,0.059804,0.023149,0.000165,0.030547,-6.121873
8,-0.096293,-5.132905,-0.977297,0.001279,0.059670,0.022722,0.000162,0.041648,-6.081014
9,-0.095844,-5.121791,-0.958192,0.001273,0.059541,0.022278,0.000160,0.033487,-6.059089


In [ ]:
df

In [65]:
df_afolu_trajectories = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))
df_afolu_trajectories.shape                      
#self.model_attributes.get_standard_variables(df_afolu_trajectories, self.modvar_lvst_animal_weight, True, "array_base", expand_to_all_cats = True)                        

(36, 1578)

In [273]:
#vars_afolu = sa.model_attributes.build_varlist("Soil Management", variable_type = "input")
vars_afolu = sa.model_attributes.build_varlist("Forest", variable_type = "input")
#vars_afolu += sa.model_attributes.build_varlist("Forest", variable_type = "input")
#vars_afolu += sa.model_attributes.build_varlist("Land Use", variable_type = "input")
vars_afolu = [x for x in vars_afolu if (x not in df_afolu_trajectories.columns)]
vars_afolu.sort()

for k in vars_afolu:
    print(k)
    

qty_frst_biomass_consumed_by_fire_temperate_mangroves_tonne_per_ha
qty_frst_biomass_consumed_by_fire_temperate_primary_tonne_per_ha
qty_frst_biomass_consumed_by_fire_temperate_secondary_tonne_per_ha
qty_frst_biomass_consumed_by_fire_tropical_mangroves_tonne_per_ha
qty_frst_biomass_consumed_by_fire_tropical_primary_tonne_per_ha
qty_frst_biomass_consumed_by_fire_tropical_secondary_tonne_per_ha


In [409]:
print(model_afolu.modvar_lsmm_emissions_n2o)

:math:\text{N}_2\text{O} Emissions from Manure Management


In [80]:

#df_project_afolu = model_afolu.project(df_afolu_trajectories)
#df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_afolu], sa.model_attributes, "concatenate")





In [16]:
dict_arrs_lsmm_frac_manure = sa.model_attributes.get_multivariables_with_bounded_sum_by_category(
    df_afolu_trajectories,
    model_afolu.modvar_lvst_list_mm_fractions,
    1,
    force_sum_equality = True,
    msg_append = "Energy fractions by category do not sum to 1. See definition of dict_arrs_inen_frac_energy."
)

In [26]:
sa.model_attributes.get_standard_variables(df_afolu_trajectories, model_afolu.modvar_lvst_genfactor_volatile_solids)

,genfactor_lvst_daily_volatile_solids_buffalo,genfactor_lvst_daily_volatile_solids_cattle_dairy,genfactor_lvst_daily_volatile_solids_cattle_nondairy,genfactor_lvst_daily_volatile_solids_chickens,genfactor_lvst_daily_volatile_solids_goats,genfactor_lvst_daily_volatile_solids_horses,genfactor_lvst_daily_volatile_solids_mules,genfactor_lvst_daily_volatile_solids_pigs,genfactor_lvst_daily_volatile_solids_sheep
0,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
1,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
2,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
3,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
4,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
5,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
6,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
7,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
8,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
9,0.0112,0.0079,0.0085,0.0135,0.0104,0.0072,0.0072,0.005,0.0083
